In [173]:
import numpy as np
#initialising parameters of HMM
def param_init(N,M):
    A = np.zeros((N,N))
    for i in range(len(A)):
        for j in range(len(A[i])):
            if(j >= i):
                A[i][j] = 1/(len(A[i])-i)
    B = np.zeros((N,M))
    for i in range(len(B)):
        B[i] = np.ones(M)/M
    pi = np.zeros(N)
    pi[0] = 1
    return A,B,pi

In [174]:
def alpha(A,B,pi,N,O):
    alf = np.zeros((len(O),N))
    for i in range(N):
        alf[0][i] = pi[i]*B[i][int(O[0])]
    for k in range(1,len(O)):
        for j in range(N):
            temp=0
            for i in range(N):
                temp += alf[k-1][i]*A[i][j]
            alf[k][j] = temp*B[j][int(O[k])]
    return alf

In [175]:
def beeta(A,B,N,O):
    bt = np.zeros((len(O),N))
    bt[len(O)-1] = np.ones(N)
    for k in range(len(O)-2,-1,-1):
        for i in range(N):
            temp=0
            for j in range(N):
                temp += A[i][j]*bt[k+1][j]
            bt[k][i] = temp*B[i][int(O[k])]
    return bt

In [176]:
def zeeta(A,B,N,O,alf,bt):
    si = np.zeros((len(O)-1,N,N))
    temp = np.zeros((len(O)-1,N,N))
    temp1 = np.zeros(len(O)-1)
    for t in range(len(O)-1):
        for i in range(N):
            for j in range(N):
                temp[t][i][j] = alf[t][i]*A[i][j]*B[j][O[t+1]]*bt[t+1][j]
                temp1[t] += temp[t][i][j]
    for t in range(len(O)-1):
        for i in range(N):
            for j in range(N):
                si[t][i][j] = temp[t][i][j]/temp1[t]
    return si

In [177]:
def gama(N,O,alf,bt):
    gm = np.zeros((len(O),N))
    temp = np.zeros((len(O),N))
    temp1 = np.zeros(len(O))
    for t in range(len(O)):
        for i in range(N):
            temp[t][i] = alf[t][i]*bt[t][i]
            temp1[t] += temp[t][i]
    for t in range(len(O)):
        for i in range(N):
            gm[t][i] = temp[t][i]/temp1[t]
    return gm

In [178]:
def EM_method(A,B,pi,N,M,O):
    temp = np.zeros((N,N))
    temp1 = np.zeros((N,M))
    logp = 0
    for O_ in O:
        eff_trans_i_j = np.zeros((N,N))
        eff_trans_i = np.zeros(N)
        eff_tim_i = np.zeros(N)
        eff_tim_i_j = np.zeros((N,M))
        temp2 = 0
        alf = alpha(A,B,pi,N,O_)
        bt = beeta(A,B,N,O_)
        si = zeeta(A,B,N,O_,alf,bt)
        gm = gama(N,O_,alf,bt)
        for i in range(N):
            temp2 += alf[len(O_)-1][i]
        logp += np.log(temp2)
        for i in range(N):
            for j in range(N):
                for t in range(len(O_)-1):
                    eff_trans_i_j[i][j] += si[t][i][j]
        for i in range(N):
            for t in range(len(O_)):
                if(t != len(O_)-1):
                    eff_trans_i[i] += gm[t][i]
                eff_tim_i[i] += gm[t][i]
                eff_tim_i_j[i][O_[t]] += gm[t][i]
        for i in range(N):
            for j in range(N):
                temp[i][j] += eff_trans_i_j[i][j]/eff_trans_i[i]
        for i in range(N):
            for j in range(M):
                temp1[i][j] += eff_tim_i_j[i][j]/eff_tim_i[i]
    for i in range(N):
        for j in range(N):
            A[i][j] = temp[i][j]/len(O)
    for i in range(N):
        for j in range(M):
            B[i][j] = temp1[i][j]/len(O)
    return logp

In [179]:
def main_est(N,M,O):
    A,B,pi = param_init(N,M)
    count = 0
    while(count<51):
        count += 1
#         temp = 0
        logp = EM_method(A,B,pi,N,M,O)
#         if(logp-temp < 0.00001):
#             break
#         else:
#             temp = logp
    return A,B,pi

In [180]:
def prob(A,B,pi,N,M,O):
    temp2 = np.zeros(len(O))
    count = 0
    for O_ in O:
        alf = alpha(A,B,pi,N,O_)
        for i in range(N):
            temp2[count] += alf[len(O_)-1][i]
#         print(temp2[count])
#         print('****************************')
        count += 1
#     print(temp2)
#     print('##############')
    return temp2

In [181]:
def classify(A,B,pi,N,M,O):
    temp1 = np.zeros((3,len(O[0])))
    temp2 = np.zeros((3,len(O[1])))
    temp3 = np.zeros((3,len(O[2])))
    for i in range(3):
        temp1[i] = prob(A[i],B[i],pi[i],N,M,O[0])
#         print('#################################')
    for i in range(3):
        temp2[i] = prob(A[i],B[i],pi[i],N,M,O[1])
#         print('#################################')
    for i in range(3):
        temp3[i] = prob(A[i],B[i],pi[i],N,M,O[2])
#         print('#################################')
#     print(temp1)
#     print('*************')
#     print(temp2)
#     print('*************')
#     print(temp3)
#     print('*************')
    cls1 = np.zeros(len(O[0]))
    cls2 = np.zeros(len(O[1]))
    cls3 = np.zeros(len(O[2]))
    cls1 = np.argmax(temp1,axis=0)
    cls2 = np.argmax(temp2,axis=0)
    cls3 = np.argmax(temp3,axis=0)
    return cls1,cls2,cls3

In [182]:
def main(N,M,train_data,test_data):
    A = np.zeros((3,N,N))
    B = np.zeros((3,N,M))
    pi = np.zeros((3,N))
    for i in range(3):
        A[i],B[i],pi[i] = main_est(N,M,train_data[i])
    cls1 = np.zeros(len(test_data[0]))
    cls2 = np.zeros(len(test_data[1]))
    cls3 = np.zeros(len(test_data[2]))
    cls1,cls2,cls3 = classify(A,B,pi,N,M,test_data)
    return cls1,cls2,cls3

In [183]:
def get_confusion_mat(true_class, pred_class):
    conf_mat = np.full((3,3),1e-10)
    for i in range(len(true_class)):
        conf_mat[true_class[i]][pred_class[i]]+=1
    return np.array(conf_mat)

In [184]:
def get_metric(conf_matrix):
    x = conf_matrix.shape[0]
    precision = []
    accuracy = []
    recall = []
    f_measure = []
    total_sum = np.sum(conf_matrix)
    for i in range(x):
        TP = conf_matrix[i][i]
        FN = np.sum(conf_matrix[i,:])-conf_matrix[i][i]
        FP = np.sum(conf_matrix[:,i])-conf_matrix[i][i]
        TN = total_sum-(TP+FP+FN)

        recal = TP/(TP+FN)
        preci = TP/(TP+FP)
        acc = (TP+TN)/(TP+TN+FP+FN)
        
        precision.append(preci)
        recall.append(recal)
        accuracy.append(acc)
        f_measure.append((2*recal*preci)/(recal+preci))
        
    return accuracy,precision,recall,f_measure

In [185]:
if(__name__=="__main__"):
    N = [2,3,4,5]
    for n in N:
        print("#"*10,"Number of States: {}".format(n),'#'*10)
        K = [8, 16, 32, 64]
        for k in K:
            print("*"*10,"Number of Clusters: {}".format(k),'*'*10)
            train_data = np.load('train_{}.npy'.format(k))
            test_data = np.load('test_{}.npy'.format(k))
#             print(train_data)
            clas1, clas2, clas3 = main(n, k, train_data, test_data)
            print(clas1,clas2,clas3)
            tot_len = len(clas1)+len(clas2)+len(clas3)
            pred_class = []
            true_class = []
            for i in range(len(clas1)):
                pred_class.append(clas1[i])
                true_class.append(0)
            for i in range(len(clas2)):
                pred_class.append(clas2[i])
                true_class.append(1)
            for i in range(len(clas3)):
                pred_class.append(clas3[i])
                true_class.append(2)
                
            conf_mat = get_confusion_mat(true_class, pred_class)
            accuracy, precision, recall, f_measure = get_metric(conf_mat)
            
            print(conf_mat)
            print("Accuracy: ",accuracy)
            print("Precision: ",precision)
            print("Recall: ",recall)
            print("F_Measure: ",f_measure)
            
            
                
                
                
                
            
            

########## Number of States: 2 ##########
********** Number of Clusters: 8 **********
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0] [0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [0 0 0 0 2 0 2 0 0 2 2 0 0 0]
[[2.5e+01 2.0e+00 1.0e-10]
 [2.3e+01 1.0e-10 1.0e+00]
 [1.0e+01 1.0e-10 4.0e+00]]
Accuracy:  [0.46153846153976325, 0.5999999999993846, 0.8307692307654201]
Precision:  [0.43103448275811534, 4.99999999925e-11, 0.799999999972]
Recall:  [0.9259259259193415, 4.1666666666145835e-12, 0.2857142857153061]
F_Measure:  [0.5882352941158477, 7.692307692130178e-12, 0.4210526315761773]
********** Number of Clusters: 16 **********
[0 0 0 1 1 1 2 1 0 2 0 1 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0] [0 0 0 0 0 1 0 1 0 2 1 1 2 1 0 0 1 2 1 1 1 0 1 0] [2 2 0 1 2 0 2 2 0 2 2 0 2 1]
[[16.  9.  2.]
 [11. 10.  3.]
 [ 4.  2.  8.]]
Accuracy:  [0.5999999999993845, 0.6153846153837871, 0.8307692307654201]
Precision:  [0.5161290322562955, 0.4761904761884354, 0.6153846153781065]
Recall:  [0.5925925925897119, 0.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars


[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2] [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2] [2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[[1.0e-10 1.0e-10 2.7e+01]
 [1.0e-10 1.0e-10 2.4e+01]
 [1.0e-10 1.0e-10 1.4e+01]]
Accuracy:  [0.5846153846149823, 0.6307692307681894, 0.21538461538932532]
Precision:  [0.33333333333333337, 0.33333333333333337, 0.21538461538515974]
Recall:  [3.703703703662551e-12, 4.1666666666145835e-12, 0.9999999999857143]
F_Measure:  [7.407407407242798e-12, 8.333333333125e-12, 0.3544303797466752]
########## Number of States: 4 ##########
********** Number of Clusters: 8 **********
[0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0] [0 0 0 0 0 0 0 0 0 2 0 0 1 0 0 0 0 0 1 0 1 0 1 0] [2 2 0 0 2 0 1 0 2 2 2 0 2 1]
[[2.3e+01 4.0e+00 1.0e-10]
 [1.9e+01 4.0e+00 1.0e+00]
 [5.0e+00 2.0e+00 7.0e+00]]
Accuracy:  [0.5692307692305798, 0.5999999999993846, 0.8769230769186273]
Precision:  [0.48936170212666363, 0.399999999998, 0.8749999999796875]
Recall:  [0.8518518518460906, 0.16

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[[1.0e-10 2.7e+01 1.0e-10]
 [1.0e-10 2.4e+01 1.0e-10]
 [1.0e-10 1.4e+01 1.0e-10]]
Accuracy:  [0.5846153846149824, 0.36923076923334924, 0.784615384612213]
Precision:  [0.33333333333333337, 0.36923076923060355, 0.33333333333333337]
Recall:  [3.703703703662552e-12, 0.9999999999916668, 7.142857142704082e-12]
F_Measure:  [7.4074074072428e-12, 0.5393258426952405, 1.4285714285102042e-11]
